## Cloning and Using openSMILE

In [4]:
!git clone https://github.com/audeering/opensmile.git

Cloning into 'opensmile'...
remote: Enumerating objects: 1830, done.
remote: Counting objects: 100% (77/77), done.
remote: Compressing objects: 100% (54/54), done.
remote: Total 1830 (delta 24), reused 55 (delta 20), pack-reused 1753 (from 1)
Receiving objects: 100% (1830/1830), 10.78 MiB | 16.85 MiB/s, done.
Resolving deltas: 100% (693/693), done.


In [1]:
!opensmile/build/progsrc/smilextract/SMILExtract -h

 
   openSMILE version 3.0.1 (Rev. 341aea9)
   Build date: 2024-10-18T09:01:32Z
   Build branch: 'master'
   (c) 2022 by audEERING GmbH
   All rights reserved. See the file COPYING for license terms.
 
Usage: opensmile/build/progsrc/smilextract/SMILExtract [-option (value)] ...
 
 -h  Show this usage information
 
 -C, -configfile <string>
     Path to openSMILE config file
     (default: 'smile.conf')
 
 -l, -loglevel <integer>
     Verbosity level (0-9)
     (default: 2)
 
 -t, -nticks <integer>
     Number of ticks to process (-1 = infinite) (only works for single-thread processing, i.e. nThreads=1)
     (default: -1)
 
 -L, -components [boolean 0/1]
     Show component list
     (default: 0)
 
 -H, -configHelp [string]
     Show documentation of registered config types (if an argument is given, show only documentation for config types beginning with the name given in the argument)
     (default: '')
 
 -configDflt [string]
     Show default config section templates for each config 

## Loading Audio Sample

In [2]:
data_path = '../../data/'

In [3]:
import os
import numpy as np
import pandas as pd
import json

In [4]:
with open(data_path + 'train.json') as json_file:
    data = json.load(json_file)
data[0]

{'conversation_ID': 1,
 'conversation': [{'utterance_ID': 1,
   'text': 'Alright , so I am back in high school , I am standing in the middle of the cafeteria , and I realize I am totally naked .',
   'speaker': 'Chandler',
   'emotion': 'neutral',
   'video_name': 'dia1utt1.mp4'},
  {'utterance_ID': 2,
   'text': 'Oh , yeah . Had that dream .',
   'speaker': 'All',
   'emotion': 'neutral',
   'video_name': 'dia1utt2.mp4'},
  {'utterance_ID': 3,
   'text': 'Then I look down , and I realize there is a phone ... there .',
   'speaker': 'Chandler',
   'emotion': 'surprise',
   'video_name': 'dia1utt3.mp4'},
  {'utterance_ID': 4,
   'text': 'Instead of ... ?',
   'speaker': 'Joey',
   'emotion': 'surprise',
   'video_name': 'dia1utt4.mp4'},
  {'utterance_ID': 5,
   'text': 'That is right .',
   'speaker': 'Chandler',
   'emotion': 'anger',
   'video_name': 'dia1utt5.mp4'},
  {'utterance_ID': 6,
   'text': 'Never had that dream .',
   'speaker': 'Joey',
   'emotion': 'neutral',
   'video_nam

In [5]:
audio_dir = data_path+'all_videos_and_audios/train/'
os.listdir(audio_dir)[:10]

['dia573utt9.wav',
 'dia1289utt4.mp4',
 'dia671utt2.mp4',
 'dia850utt5.wav',
 'dia1212utt8.mp4',
 'dia1158utt13.mp4',
 'dia686utt1.mp4',
 'dia1032utt7.mp4',
 'dia552utt8.wav',
 'dia1204utt4.mp4']

In [6]:
ridx = np.random.randint(0, len(data))
audio_sample = audio_dir+data[ridx]['conversation'][0]['video_name'].split('.')[0] + '.wav'
audio_sample

'../../data/all_videos_and_audios/train/dia475utt1.wav'

In [7]:
from IPython.display import Audio
print(data[ridx]['conversation'][0]['text'])
Audio(audio_sample)

... and then Zana , just let one rip ! !


## Extract embeddings

In [8]:
import subprocess

In [9]:
def extract_features(audio_file_path, config_file_path = 'opensmile/config/is09-13/IS13_ComParE.conf'):

    out = audio_file_path.split('/')[-1].replace('.wav', '_features.txt')
    output_file_path = 'features/'+out

    # outfiles = os.listdir('features/')
    # if out in outfiles:
    #     return np.loadtxt('features/'+out, delimiter=',')
    
    command = [
        'opensmile/build/progsrc/smilextract/SMILExtract',
        '-C', config_file_path,
        '-I', audio_file_path,
        '-O', output_file_path
    ]
    
    subprocess.run(command, check=True)
    
    raw_features = []
    with open(output_file_path) as f:
        for line in f:
            if line.startswith('@') or line=='\n':
                continue
            raw_features = line.strip().split(',')
            raw_features = [i for i in raw_features[1:-1]]

    return raw_features

In [11]:
sample_features = extract_features(audio_sample)

(MSG) [2] SMILExtract: openSMILE starting!
(MSG) [2] SMILExtract: config file is: opensmile/config/is09-13/IS13_ComParE.conf
(MSG) [2] cComponentManager: successfully registered 102 component types.
(MSG) [2] instance 'lldsink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldhtksink': No filename given, disabling this sink component.
(MSG) [2] instance 'lldarffsink': No filename given, disabling this sink component.
(MSG) [2] instance 'csvsink': No filename given, disabling this sink component.
(MSG) [2] instance 'htksink': No filename given, disabling this sink component.
(MSG) [2] instance 'arffsink': writing ARFF header (6373 features)...
(MSG) [2] instance 'arffsink': finished writing ARFF header.
(MSG) [2] cComponentManager: successfully finished createInstances (48 component instances were finalised, 1 data memories were finalised)
(MSG) [2] cComponentManager: starting single thread processing loop
(WRN) [2] cComponentManager: The following component(s) 

In [12]:
print(len(sample_features))
sample_features[:10]

6373


['1.443101e+00',
 '7.134671e-01',
 '4.985673e-01',
 '2.970338e-01',
 '5.424609e-01',
 '8.340309e-01',
 '2.454270e-01',
 '2.915700e-01',
 '5.369971e-01',
 '1.987813e-01']

In [23]:
import numpy as np

In [24]:
features_array = np.load('audio_features.npy')
print(features_array.shape)

(3, 6373)


In [29]:
print(features_array[0][:10])

[1.396228  0.2410714 0.9910714 0.2919344 0.5737295 0.853347  0.2817951
 0.2796175 0.5614126 0.163341 ]
